## PART **1**

In [ ]:
## Install tensorflow recommenders and datasets to our current environment

!pip install -q tensorflow-recommenders &> /dev/null
!pip install -q --upgrade tensorflow-datasets &> /dev/null

In [ ]:
## Import our libraries

from typing import Dict, Text
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs
import pprint

In [ ]:
# Ratings data.
ratings = tfds.load("movielens/100k-ratings", split="train")

In [ ]:
## Let's look at our rating data

for x in ratings.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'bucketized_user_age': 45.0,
 'movie_genres': array([7]),
 'movie_id': b'357',
 'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
 'raw_user_age': 46.0,
 'timestamp': 879024327,
 'user_gender': True,
 'user_id': b'138',
 'user_occupation_label': 4,
 'user_occupation_text': b'doctor',
 'user_rating': 4.0,
 'user_zip_code': b'53211'}


In [ ]:
## We only need to use movie_title, user_id, and user_rating to build our model

ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"]
})

## Take a look at our new dataset
for x in ratings.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
 'user_id': b'138',
 'user_rating': 4.0}


In [ ]:
## Split data into train and test - 80% train - 20% test

shuffled = ratings.shuffle(100000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80000)
test = shuffled.skip(80000).take(20000)

In [ ]:
## Map our unique user ids and movie titles

## takes only relevant columns
movie_titles = ratings.batch(100000).map(lambda x: x["movie_title"])
user_ids = ratings.batch(100000).map(lambda x: x["user_id"])

## takes only unique values in those columns
unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [ ]:
print(unique_movie_titles[:5])
print('\t')
print(f'number of unique movies: ' + str(len(unique_movie_titles)))

[b"'Til There Was You (1997)" b'1-900 (1994)' b'101 Dalmatians (1996)'
 b'12 Angry Men (1957)' b'187 (1997)']
	
number of unique movies: 1664


In [ ]:
print(unique_user_ids[:5])
print('\t')
print(f'number of unique user ids: ' + str(len(unique_user_ids)))

[b'1' b'10' b'100' b'101' b'102']
	
number of unique user ids: 943


In [ ]:
## Implementing a model

class RankingModel(tf.keras.Model):

  def __init__(self):
    super().__init__()
    embedding_dimension = 32

    # A layer that uses user ids as inputs and converts them into embeddings
    self.user_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # A layer that uses movie titles as inputs and converts them into embeddings
    self.movie_embeddings = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
        vocabulary=unique_movie_titles, mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
    ])

    # A sequence of dense layers that take the concatenation of a user id and movie title and makes a prediction of the movie rating as an output
    self.ratings = tf.keras.Sequential([
      # Learn multiple dense layers.
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      # Make rating predictions in the final layer.
      tf.keras.layers.Dense(1)
  ])

  # The model takes user id and movie title and passes them through the layers above to obtain a predicted movie rating
  def call(self, inputs):

    user_id, movie_title = inputs

    user_embedding = self.user_embeddings(user_id)
    movie_embedding = self.movie_embeddings(movie_title)

    return self.ratings(tf.concat([user_embedding, movie_embedding], axis=1))

In [ ]:
RankingModel()((["42"], ["One Flew Over the Cuckoo's Nest (1975)"]))

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.03950961]], dtype=float32)>

## **PART 2**

In [ ]:
## Build the loss used to train our model
## We are using the Ranking task object and MSE for our loss function
## This task returns the computed loss, 
## which we can use to implement in our model's training loop

task = tfrs.tasks.Ranking(
  loss = tf.keras.losses.MeanSquaredError(),
  metrics=[tf.keras.metrics.RootMeanSquaredError()]
)

In [ ]:
## Build our full model
## Use the task created above to minimize our losses

class MovielensModel(tfrs.models.Model):

  def __init__(self):
    super().__init__()
    ## Create a RankingModel instance
    self.ranking_model: tf.keras.Model = RankingModel()
    ## Create a loss calculation task
    self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
      loss = tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

  ## Takes a dictionary of input features (user ids, movie titles, user ratings) and returns
  ## the rating prediction made by the RankingModel
  def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
    return self.ranking_model(
        (features["user_id"], features["movie_title"]))

  ## Calculates the loss and the metrics based on the inputs
  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    labels = features.pop("user_rating")

    rating_predictions = self(features)

    return self.task(labels=labels, predictions=rating_predictions)

In [ ]:
## Fitting and Evaluating

model = MovielensModel()
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [ ]:
## Shuffle to introduce randomness
## Batch to improve efficiency/accuracy
## Cache increases the speed of the training time

cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [ ]:
model.fit(cached_train, epochs=3)

## As you can see, MSE and loss is falling as we continue to fit the model

Epoch 1/3
10/10 [==============================] - 7s 44ms/step - root_mean_squared_error: 2.1869 - loss: 4.3895 - regularization_loss: 0.0000e+00 - total_loss: 4.3895
Epoch 2/3
10/10 [==============================] - 0s 40ms/step - root_mean_squared_error: 1.1156 - loss: 1.2426 - regularization_loss: 0.0000e+00 - total_loss: 1.2426
Epoch 3/3
10/10 [==============================] - 0s 34ms/step - root_mean_squared_error: 1.1048 - loss: 1.2180 - regularization_loss: 0.0000e+00 - total_loss: 1.2180


In [ ]:
## Evaluate the model on our cached test dataset
## The lower the RMSE metric, the more accurate the model is

model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 4s 16ms/step - root_mean_squared_error: 1.1040 - loss: 1.2224 - regularization_loss: 0.0000e+00 - total_loss: 1.2224


{'root_mean_squared_error': 1.103978157043457,
 'loss': 1.2383575439453125,
 'regularization_loss': 0,
 'total_loss': 1.2383575439453125}

In [ ]:
## Testing the ranking model
## Movies would be derived from retrieval task

test_ratings = {}
test_movie_titles = ["M*A*S*H (1970)", "Dances with Wolves (1990)", "Speed (1994)"]
for movie_title in test_movie_titles:
  test_ratings[movie_title] = model({
      "user_id": np.array(["42"]),
      "movie_title": np.array([movie_title])
  })

print("Ratings:")
for title, score in sorted(test_ratings.items(), key=lambda x: x[1], reverse=True):
  print(f"{title}: {score}")

Ratings:
M*A*S*H (1970): [[3.605894]]
Speed (1994): [[3.5313191]]
Dances with Wolves (1990): [[3.5234118]]
